**Pedro Antonio Mora Chacón** *Student#* 22203184

### Prepare USA beer production by year (2008-2019) dataset for exploration. <br>

##### The data itself comes from the Alcohol and Tobacco Tax and Trade Bureau (TTB) as presented in the rfordatascience / tidytuesday repository found here; https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-03-31/readme.md.

##### Multiple CSV files will be uploaded into Pandas to cleanup, and prepare to make a combined CSV file to use in Vega-Lite.  

In [301]:
# Import modules
import pandas as pd
from sklearn.impute import KNNImputer
import numpy as np

##### Datasets:

In [302]:
# Datasets:

# beer_taxed.csv:

beer_taxed_csv = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/5fd005949175ace3551f3f7488c4685d6e48a2ee/data/2020/2020-03-31/beer_taxed.csv'

# beer_states.csv:

beer_states_csv = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/5fd005949175ace3551f3f7488c4685d6e48a2ee/data/2020/2020-03-31/beer_states.csv'

# brewer_size.csv:

brewer_size_csv = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/5fd005949175ace3551f3f7488c4685d6e48a2ee/data/2020/2020-03-31/brewer_size.csv'

# brewing_materials.csv:

brewing_materials_csv = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/5fd005949175ace3551f3f7488c4685d6e48a2ee/data/2020/2020-03-31/brewing_materials.csv'

# brewing_products.csv:

brewing_products_csv = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/5fd005949175ace3551f3f7488c4685d6e48a2ee/data/2020/2020-03-31/brewing_products.csv'

##### Dataframes:

In [303]:
# Dataframes:

# Create list to add dfs to for easier processing:
df_dict = dict()

# beer_taxed.csv:

df_dict["beer_taxed"] = pd.read_csv(beer_taxed_csv,  keep_default_na=True, delimiter=',', skipinitialspace=True)


# beer_states.csv:

df_dict["beer_states"] = pd.read_csv(beer_states_csv,  keep_default_na=True, delimiter=',', skipinitialspace=True)


# brewer_size.csv:

df_dict["brewer_size"] = pd.read_csv(brewer_size_csv,  keep_default_na=True, delimiter=',', skipinitialspace=True)


# brewing_materials.csv:

df_dict["brewing_materials"] = pd.read_csv(brewing_products_csv,  keep_default_na=True, delimiter=',', skipinitialspace=True)


# brewing_products.csv:

df_dict["brewing_products"] = pd.read_csv(brewing_materials_csv,  keep_default_na=True, delimiter=',', skipinitialspace=True)


##### Looking briefly at each set  

In [304]:
# Getting more info on each df

# Function to print column data types
def df_dtypes(name, df):
    print(f"\n{df.dtypes}\n",)

# Function to print categorical column data values
def df_cat_values(name, df):
    for column in df.select_dtypes(['object']).columns:
        print(f"\n\ndf_dict['{name}']['{column}'] values:")
        print(df[column].value_counts(dropna=False))
    
# Function to print first 3 lines
def head3(name, df):
    print(f"\n{df.head(3)}\n",)

# Run each df

for k,v in df_dict.items():
    print("*"*100,f"\ndf_dict['{k}']\n".upper(),"*"*100,"\n\n")
    # get column data types
    df_dtypes(k,v)
    print(" "*5,"-"*60,'\n')
    # sort by year to get earliest and latest for each df
    v.sort_values(by='year', inplace=True, kind='mergesort', ignore_index=True)
    # print earliest and latest year
    df_year = [x for x in v['year']]
    print(" "*5,f"df_dict['{k}']: earliest = {df_year[0]} most recent = {df_year[-1]}")
    print(" "*5,"-"*60,'\n')
    print(f"df_dict['{k}'] categorical column data values:\n")
    df_cat_values(k,v)
    print(" "*5,"-"*60,'\n')
    # head3(k,v)



**************************************************************************************************** 
DF_DICT['BEER_TAXED']
 **************************************************************************************************** 



data_type            object
tax_status           object
year                  int64
month                 int64
type                 object
month_current         int64
month_prior_year      int64
ytd_current         float64
ytd_prior_year      float64
tax_rate             object
dtype: object

      ------------------------------------------------------------ 

      df_dict['beer_taxed']: earliest = 2008 most recent = 2019
      ------------------------------------------------------------ 

df_dict['beer_taxed'] categorical column data values:



df_dict['beer_taxed']['data_type'] values:
data_type
Barrels Produced    1580
Name: count, dtype: int64


df_dict['beer_taxed']['tax_status'] values:
tax_status
Totals                432
Taxable               432
Tax

##### Reviewing the csv files, brewing_materials.csv and brewing_products.csv looked like the same data. Checking to see if so.

In [305]:
print(f"Are the brewing_materials.csv and brewing_products.csv the \nsame dataset with different names?:  {df_dict['brewing_products'].equals(df_dict['brewing_materials'])}.")

Are the brewing_materials.csv and brewing_products.csv the 
same dataset with different names?:  True.


##### Dropping duplicate dataframe.


In [306]:
df_dict.pop("brewing_products")

,data_type,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
0,Pounds of Materials Used,Grain Products,2008,1,Malt and malt products,374165152,365300134,374165152.0,365300134.0
1,Pounds of Materials Used,Grain Products,2008,1,Corn and corn products,57563519,41647092,57563519.0,41647092.0
2,Pounds of Materials Used,Grain Products,2008,1,Rice and rice products,72402143,81050102,72402143.0,81050102.0
3,Pounds of Materials Used,Grain Products,2008,1,Barley and barley products,3800844,2362162,3800844.0,2362162.0
4,Pounds of Materials Used,Grain Products,2008,1,Wheat and wheat products,1177186,1195381,1177186.0,1195381.0
...,...,...,...,...,...,...,...,...,...
1435,Pounds of Materials Used,Non-Grain Products,2017,12,Hops (dry),1453024,3040143,NaN,NaN
1436,Pounds of Materials Used,Non-Grain Products,2017,12,Hops (used as extracts),2117,29141,NaN,NaN
1437,Pounds of Materials Used,Non-Grain Products,2017,12,Other,657306,4048091,NaN,NaN
1438,Pounds of Materials Used,Total Non-Grain products,2017,12,Total Non-Grain products,2148660,7759483,NaN,NaN


##### First we will look at null Values in each df. 

In [307]:
# Create function to look at each df for null values
def nullsums(d):
    for name, df in d.items():
        print(name+".csv:\n","_"*100,f"\n{df.isnull().sum()}\n","_"*100,"\n\n")

# Run each df

nullsums(df_dict)

beer_taxed.csv:
 ____________________________________________________________________________________________________ 
data_type            0
tax_status           0
year                 0
month                0
type                 0
month_current        0
month_prior_year     0
ytd_current         24
ytd_prior_year      24
tax_rate             0
dtype: int64
 ____________________________________________________________________________________________________ 


beer_states.csv:
 ____________________________________________________________________________________________________ 
state       0
year        0
barrels    19
type        0
dtype: int64
 ____________________________________________________________________________________________________ 


brewer_size.csv:
 ____________________________________________________________________________________________________ 
year                0
brewer_size         0
n_of_brewers        0
total_barrels       1
taxable_removals    0
total_shi

##### There are constant columns as we saw above. They do not bring value to the visualization so they are drop from the datasets 

In [308]:
# Dropping beer_taxed_df['data_type'] which only says 'Barrels Produced'. brewing_materials df_dict['brewing_materials']
df_dict['beer_taxed'] = df_dict['beer_taxed'].drop("data_type", axis='columns')
print("Columns after removing 'data_type' constant columns",'\n',[df_dict['beer_taxed'].columns])

# Dropping brewing_materials_df['data_type'] which only says 'Pounds of Materials Used'.
df_dict['brewing_materials'] = df_dict['brewing_materials'].drop("data_type", axis='columns')
print("Columns after removing 'data_type' constant columns",'\n',[df_dict['brewing_materials'].columns])

Columns after removing 'data_type' constant columns 
 [Index(['tax_status', 'year', 'month', 'type', 'month_current',
       'month_prior_year', 'ytd_current', 'ytd_prior_year', 'tax_rate'],
      dtype='object')]
Columns after removing 'data_type' constant columns 
 [Index(['material_type', 'year', 'month', 'type', 'month_current',
       'month_prior_year', 'ytd_current', 'ytd_prior_year'],
      dtype='object')]


##### The null values appear in the 'Year To Date' cumulative type columns for the beer_taxed.csv and brewing_materials.csv dataframes. The data we will use to make the visualizations is the current year, non-cumulative type data in order to be able to present information at different hierarchy levels through aggregations. Therefore, 'Prior Year' or 'Year To Date' cumulative type columns are not going to be used in the graphs and can safely be removed from the dataframes. 


In [309]:
# Create function to look at each df and drop cumulative type columns
print('*'*100,'\n\n')
def drop_ytd_prior_col(d):
    for name, df in d.items():
        # print csv dataset
        print(f"{name}.csv:\n","_"*100)
        # print Original columns 
        print("Columns before removing 'ytd' & 'prior' columns",'\n',[df.columns])
        # Remove columns with 'ytd' or 'prior'
        df = df.filter(regex='^(?!.*ytd).*$', axis=1)
        df = df.filter(regex='^(?!.*prior).*$', axis=1)
        d[name] = df
        # print columns after
        print("Columns after removing 'ytd' & 'prior' columns",'\n',[df.columns])
        print('*'*100,'\n\n')

# Run each df

drop_ytd_prior_col(df_dict)

**************************************************************************************************** 


beer_taxed.csv:
 ____________________________________________________________________________________________________
Columns before removing 'ytd' & 'prior' columns 
 [Index(['tax_status', 'year', 'month', 'type', 'month_current',
       'month_prior_year', 'ytd_current', 'ytd_prior_year', 'tax_rate'],
      dtype='object')]
Columns after removing 'ytd' & 'prior' columns 
 [Index(['tax_status', 'year', 'month', 'type', 'month_current', 'tax_rate'], dtype='object')]
**************************************************************************************************** 


beer_states.csv:
 ____________________________________________________________________________________________________
Columns before removing 'ytd' & 'prior' columns 
 [Index(['state', 'year', 'barrels', 'type'], dtype='object')]
Columns after removing 'ytd' & 'prior' columns 
 [Index(['state', 'year', 'barrels', 'type

##### Lets look at the other column cases of null values to see in closer detail what we are dealing with.  

In [310]:
# Create function to look at each for null values
def nullrows(d):
    for name, df in d.items():
        print(f"{name}.csv:\n","_"*100)
        for c in df.columns:
            if df[c].isnull().sum() > 0:
                print(f"{name}_df['{c}'] nulls:\n {df[df[c].isnull()]}")

# Run each df

nullrows(df_dict)

beer_taxed.csv:
 ____________________________________________________________________________________________________
beer_states.csv:
 ____________________________________________________________________________________________________
beer_states_df['barrels'] nulls:
     state  year  barrels              type
25     MS  2008      NaN       On Premises
53     AL  2008      NaN  Bottles and Cans
80     ND  2008      NaN  Bottles and Cans
93     SD  2008      NaN  Bottles and Cans
111    DC  2008      NaN  Kegs and Barrels
132    ND  2008      NaN  Kegs and Barrels
209    AL  2009      NaN  Bottles and Cans
236    ND  2009      NaN  Bottles and Cans
249    SD  2009      NaN  Bottles and Cans
267    DC  2009      NaN  Kegs and Barrels
288    ND  2009      NaN  Kegs and Barrels
365    AL  2010      NaN  Bottles and Cans
392    ND  2010      NaN  Bottles and Cans
423    DC  2010      NaN  Kegs and Barrels
444    ND  2010      NaN  Kegs and Barrels
548    ND  2011      NaN  Bottles and Can

##### For the beer_states_df['barrels'] with null values we are using the scikit-learn KNNImputer (for neighbours = 3) to impute the 'barrels' column null values. First though, we will need to make sure that if there are any row subtotals, they are removed before training the model. 

In [311]:
# Remove any subtotal rows 
df_dict['beer_states'] = df_dict['beer_states'][~df_dict['beer_states']['state'].astype(str).str.contains('total', case=False)]

# KNNImputer with neighbors=3
knn_imputer = KNNImputer(n_neighbors=3)

# fill the missing values in the 'barrels' column
df_dict['beer_states']['barrels'] = knn_imputer.fit_transform(df_dict['beer_states'][['barrels']])



##### For the df 'brewer_size' column 'total_barrels', one null value we are using the mean of the 'Under 1 Barrel' category.


In [312]:
print(f"The mean of the 'total_barrels' where 'brewer_size' column rows equal 'Under 1 Barrel', excluding where 'total_barrels' is null = {round(df_dict['brewer_size'].loc[df_dict['brewer_size']['brewer_size'] == 'Under 1 Barrel', 'total_barrels'].dropna().mean(),2)}")


The mean of the 'total_barrels' where 'brewer_size' column rows equal 'Under 1 Barrel', excluding where 'total_barrels' is null = 8.13


In [313]:
# Replacing one null value in  df_dict['brewer_size']['total_barrels']
df_dict['brewer_size']['total_barrels'] = df_dict['brewer_size']['total_barrels'].fillna(round(df_dict['brewer_size'].loc[df_dict['brewer_size']['brewer_size'] == 'Under 1 Barrel', 'total_barrels'].dropna().mean(),2))
print(f"df nulls:\n {df_dict['brewer_size'][df_dict['brewer_size']['total_barrels'].isnull()]}",'-'*50)
print(df_dict['brewer_size'].loc[135])

df nulls:
 Empty DataFrame
Columns: [year, brewer_size, n_of_brewers, total_barrels, taxable_removals, total_shipped]
Index: [] --------------------------------------------------
year                          2019
brewer_size         Under 1 Barrel
n_of_brewers                   174
total_barrels                 8.13
taxable_removals         114960.27
total_shipped                185.0
Name: 135, dtype: object


##### For the df 'brewer_size' column 'total_shipped' with null values, rows 82 & 134, we are using the mean as well.    

In [314]:
print(f"The mean of the 'total_shipped' where 'brewer_size' column rows equal 'Zero Barrels', excluding where 'total_shipped' is null = {round(df_dict['brewer_size'].loc[df_dict['brewer_size']['brewer_size'] == 'Zero Barrels', 'total_shipped'].dropna().mean(),2)}")

print(f"The mean of the 'total_shipped' where 'brewer_size' column rows equal 'Under 1 Barrel', excluding where 'total_shipped' is null = {round(df_dict['brewer_size'].loc[df_dict['brewer_size']['brewer_size'] == 'Under 1 Barrel', 'total_shipped'].dropna().mean(),2)}")

The mean of the 'total_shipped' where 'brewer_size' column rows equal 'Zero Barrels', excluding where 'total_shipped' is null = nan
The mean of the 'total_shipped' where 'brewer_size' column rows equal 'Under 1 Barrel', excluding where 'total_shipped' is null = 336.4


##### 'Zero Barrels' returned a NaN value. If that is the only row across the years, we will use zero as probably expected for the 'Zero Barrels' category. 

In [315]:
df_dict['brewer_size'].loc[df_dict['brewer_size']['brewer_size'] == 'Zero Barrels']

,year,brewer_size,n_of_brewers,total_barrels,taxable_removals,total_shipped
82,2015,Zero Barrels,476,0.0,87811.22,NaN


##### Replacing one null value in df 'brewer_size' column 'total_shipped' in the 'brewer_size' category 'Zero Barrels'. Then checking all null values are taken care off.   

In [316]:
# Replacing one null value in  df_dict['brewer_size']['total_shipped']

df_dict['brewer_size'].iloc[82, 5] = 0

df_dict['brewer_size'].iloc[134, 5] = round(df_dict['brewer_size'].loc[df_dict['brewer_size']['brewer_size'] == 'Under 1 Barrel', 'total_shipped'].dropna().mean(),2)

# Double checking for nulls in 'total_shipped'
print(f"df nulls:\n {df_dict['brewer_size'][df_dict['brewer_size']['total_shipped'].isnull()]}",'\n','-'*50)
print(df_dict['brewer_size'].loc[82],'\n','^'*50)
print(df_dict['brewer_size'].loc[134])

# Look at any null values anywhere again
print('^'*50,'\nLook at any null values left anywhere'.upper(),'\n')
nullrows(df_dict)

df nulls:
 Empty DataFrame
Columns: [year, brewer_size, n_of_brewers, total_barrels, taxable_removals, total_shipped]
Index: [] 
 --------------------------------------------------
year                        2015
brewer_size         Zero Barrels
n_of_brewers                 476
total_barrels                0.0
taxable_removals        87811.22
total_shipped                0.0
Name: 82, dtype: object 
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
year                          2019
brewer_size         Under 1 Barrel
n_of_brewers                    28
total_barrels                 18.3
taxable_removals          19432.26
total_shipped                336.4
Name: 134, dtype: object
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 
LOOK AT ANY NULL VALUES LEFT ANYWHERE 

beer_taxed.csv:
 ____________________________________________________________________________________________________
beer_states.csv:
 _________________________________________________________________________________


##### It was also observed that there are 'Total' type rows that contain subtotals based on the time period term. To see if nulls are present in the granular data or the subtotals, we will need to look at the rows with null values. As mentioned, the visualizations will use granular data and these subtotals will be removed in order to avoid double counting the data figures. 

In [317]:
#  look at each df and drop cumulative type rows  df_dict['brewing_materials']
       
# brewing_materials_df
# print csv dataset
print('*'*80,"\nbrewing_materials_df:\n")
print(f"\nRows before removing subtotal rows: {df_dict['brewing_materials'].shape[0]}\n")
for c in df_dict['brewing_materials'].columns:
    if df_dict['brewing_materials'][df_dict['brewing_materials'][c].astype(str).str.contains('total', case=False)].shape[0] > 0:
        print(f"\nsample of subtotal labels in '{c}' column:\n")
        print(df_dict['brewing_materials'][df_dict['brewing_materials'][c].astype(str).str.contains('total', case=False)][c].head(4))
        df_dict['brewing_materials'] = df_dict['brewing_materials'][~df_dict['brewing_materials'][c].astype(str).str.contains('total', case=False)]            
print(f"\nRows after removing any subtotal rows: {df_dict['brewing_materials'].shape[0]}\n")

# brewer_size_df
# print csv dataset
print('*'*80,"\nbrewer_size_df:\n")
print(f"\nRows before removing subtotal rows: {df_dict['brewer_size'].shape[0]}\n")
for c in df_dict['brewer_size'].columns:
    if df_dict['brewer_size'][df_dict['brewer_size'][c].astype(str).str.contains('total', case=False)].shape[0] > 0:
        print(f"\nsample of subtotal labels in '{c}' column:\n")
        print(df_dict['brewer_size'][df_dict['brewer_size'][c].astype(str).str.contains('total', case=False)][c].head(4))
        df_dict['brewer_size'] = df_dict['brewer_size'][~df_dict['brewer_size'][c].astype(str).str.contains('total', case=False)]            
print(f"\nRows after removing any subtotal rows: {df_dict['brewer_size'].shape[0]}\n")


# beer_states_df
# print csv dataset
print('*'*80,"\nbeer_states_df:\n")
print(f"\nRows before removing subtotal rows: {df_dict['beer_states'].shape[0]}\n")
for c in df_dict['beer_states'].columns:
    if df_dict['beer_states'][df_dict['beer_states'][c].astype(str).str.contains('total', case=False)].shape[0] > 0:
        print(f"\nsample of subtotal labels in '{c}' column:\n")
        print(df_dict['beer_states'][df_dict['beer_states'][c].astype(str).str.contains('total', case=False)][c].head(4))
        df_dict['beer_states'] = df_dict['beer_states'][~df_dict['beer_states'][c].astype(str).str.contains('total', case=False)]            
print(f"\nRows after removing any subtotal rows: {df_dict['beer_states'].shape[0]}\n")


# beer_taxed_df
# print csv dataset
print('*'*80,"\nbeer_taxed_df:\n")
print(f"\nRows before removing subtotal rows: {df_dict['beer_taxed'].shape[0]}\n")
for c in df_dict['beer_taxed'].columns:
    if df_dict['beer_taxed'][df_dict['beer_taxed'][c].astype(str).str.contains('total', case=False)].shape[0] > 0:
        print(f"\nsample of subtotal labels in '{c}' column:\n")
        print(df_dict['beer_taxed'][df_dict['beer_taxed'][c].astype(str).str.contains('total', case=False)][c].head(4))
        df_dict['beer_taxed'] = df_dict['beer_taxed'][~df_dict['beer_taxed'][c].astype(str).str.contains('total', case=False)]            
print(f"\nRows after removing any subtotal rows: {df_dict['beer_taxed'].shape[0]}\n")


******************************************************************************** 
brewing_materials_df:


Rows before removing subtotal rows: 1440


sample of subtotal labels in 'material_type' column:

5         Total Grain products
10    Total Non-Grain products
11                  Total Used
17        Total Grain products
Name: material_type, dtype: object

Rows after removing any subtotal rows: 1080

******************************************************************************** 
brewer_size_df:


Rows before removing subtotal rows: 137


sample of subtotal labels in 'brewer_size' column:

11    Total
22    Total
34    Total
46    Total
Name: brewer_size, dtype: object

Rows after removing any subtotal rows: 126

******************************************************************************** 
beer_states_df:


Rows before removing subtotal rows: 1836


Rows after removing any subtotal rows: 1836

******************************************************************************** 
be

##### There are overlapping row categories for the Brewer size column as we saw above. They would mislead the visualization audience so all the categories between 1,000,000 to 6,000,000 Barrels are combined to the '1,000,000 to 6,000,000 Barrels' row category.

##### Also using this opportunity to consolidate some of the other categories. For example, the 'Under 1 Barrel' and 'Zero Barrels' in annual production could both be cases of either breweries that are in planning stages, or breweries that have ceased production. These two are similar special circumstances categories to themselves, so we are consolidating them. The remaining Categories from 1 to 15,000 and 15,001 to 999,999 are set as the last two other categories.



In [318]:
# clean up overlapping category for Brewer size df_dict['brewer_size']
rules = [
    (df_dict['brewer_size']['brewer_size'].str.contains('6,000,001 Barrels and Over', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('2,000,000 to 6,000,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('1,000,001 to 6,000,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('1,000,001 to 1,999,999 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('1,000,000 to 6,000,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('500,001 to 1,000,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('100,001 to 500,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('60,001 to 100,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('30,001 to 60,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('15,001 to 30,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('7,501 to 15,000 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('1,001 to 7,500 Barrels', case=False, flags=0, na=None, regex=False)),
    (df_dict['brewer_size']['brewer_size'].str.contains('1 to 1,000 Barrels', case=False, flags=0, na=None, regex=False))
]
# Choices
choices = [
    '6,000,001 Barrels and Over',
    '1,000,000 to 6,000,000 Barrels',
    '1,000,000 to 6,000,000 Barrels',
    '1,000,000 to 6,000,000 Barrels',
    '1,000,000 to 6,000,000 Barrels',
    '15,001 to 999,999 Barrels',
    '15,001 to 999,999 Barrels',
    '15,001 to 999,999 Barrels',
    '15,001 to 999,999 Barrels',
    '15,001 to 999,999 Barrels',
    '1 to 15,000 Barrels',
    '1 to 15,000 Barrels',
    '1 to 15,000 Barrels'
]
# Create new  'brewer_sizes' column
df_dict['brewer_size']['brewer_sizes'] = np.select(rules, choices, default='Under 1 Barrel')
# Making sure the only differences were to the overlapping categories before assigning to  'brewer_size' column
df_diff = df_dict['brewer_size'][df_dict['brewer_size']['brewer_sizes'] != df_dict['brewer_size']['brewer_size']]
print(df_diff[['brewer_size','brewer_sizes']])

                        brewer_size                    brewer_sizes
1    1,000,001 to 6,000,000 Barrels  1,000,000 to 6,000,000 Barrels
2      500,001 to 1,000,000 Barrels       15,001 to 999,999 Barrels
3        100,001 to 500,000 Barrels       15,001 to 999,999 Barrels
4         60,001 to 100,000 Barrels       15,001 to 999,999 Barrels
5          30,001 to 60,000 Barrels       15,001 to 999,999 Barrels
..                              ...                             ...
129        30,001 to 60,000 Barrels       15,001 to 999,999 Barrels
130        15,001 to 30,000 Barrels       15,001 to 999,999 Barrels
131         7,501 to 15,000 Barrels             1 to 15,000 Barrels
132          1,001 to 7,500 Barrels             1 to 15,000 Barrels
133              1 to 1,000 Barrels             1 to 15,000 Barrels

[108 rows x 2 columns]


##### Checking the Brewer size categories do not include the ones expected to be absorbed by the ones set above. 

In [319]:
df_dict['brewer_size'][['brewer_sizes','brewer_size']].groupby(['brewer_sizes']).value_counts()

brewer_sizes                    brewer_size                   
1 to 15,000 Barrels             1 to 1,000 Barrels                11
                                1,001 to 7,500 Barrels            11
                                7,501 to 15,000 Barrels           11
1,000,000 to 6,000,000 Barrels  1,000,001 to 1,999,999 Barrels     9
                                2,000,000 to 6,000,000 Barrels     9
                                1,000,000 to 6,000,000 Barrels     1
                                1,000,001 to 6,000,000 Barrels     1
15,001 to 999,999 Barrels       100,001 to 500,000 Barrels        11
                                15,001 to 30,000 Barrels          11
                                30,001 to 60,000 Barrels          11
                                500,001 to 1,000,000 Barrels      11
                                60,001 to 100,000 Barrels         11
6,000,001 Barrels and Over      6,000,001 Barrels and Over        11
Under 1 Barrel                  Under 1 

##### Checking the column totals add up correctly before finalizing 

In [320]:
# Assigning to  'brewer_size' column
df_dict['brewer_size']['brewer_size'] = df_dict['brewer_size']['brewer_sizes']

# Dropping additional 'brewer_sizes' column
df_dict['brewer_size'] = df_dict['brewer_size'].drop("brewer_sizes", axis='columns')

# summing the categories by Grouping 
df_grouped = df_dict['brewer_size'].groupby(['year', 'brewer_size']).sum().reset_index()

# Checking to see any difference between Grouped df and original for 'n_of_brewers'
n_of_brewers_sum =  round(df_dict['brewer_size']['n_of_brewers'].sum() - df_grouped['n_of_brewers'].sum(),2)

# Checking to see any difference between Grouped df and original for 'total_barrels'
total_barrels_sum = round(df_dict['brewer_size']['total_barrels'].sum() - df_grouped['total_barrels'].sum(),2)

# Checking to see any difference between Grouped df and original for 'taxable_removals'
taxable_removals_sum = round(df_dict['brewer_size']['taxable_removals'].sum() - df_grouped['taxable_removals'].sum(),2)

# Checking to see any difference between Grouped df and original for 'total_shipped'
total_shipped_sum = round(df_dict['brewer_size']['total_shipped'].sum() - df_grouped['total_shipped'].sum(),2)

print(f"Difference between 'n_of_brewers': {n_of_brewers_sum:,}")
print(f"Difference between 'total_barrels': {total_barrels_sum:,}")
print(f"Difference between 'taxable_removals': {taxable_removals_sum:,}")
print(f"Difference between 'total_shipped': {total_shipped_sum:,}")

Difference between 'n_of_brewers': 0
Difference between 'total_barrels': 0.0
Difference between 'taxable_removals': -0.0
Difference between 'total_shipped': 0.0


##### Setting 'brewer_size' dataframe in df dictionary to new categories consolidated version. 

In [321]:
# Setting 'brewer_size' in df dictionary to new categories version. 
df_dict['brewer_size'] = df_grouped

##### In trying to make a decision whether the 'Under 1 Barrel' (special circumstances) category adds  significant value to the  visualization, we are checking to see how often is the 'Under 1 Barrel' category represented in the dataset by looking at the categories present annually in the dataset ('brewer_size' dataframe in df dictionary). 

In [322]:
# df_dict['brewer_size'][['brewer_size']].groupby(['brewer_size']).value_counts()
df_dict['brewer_size']['brewer_size'].value_counts()

brewer_size
1 to 15,000 Barrels               11
1,000,000 to 6,000,000 Barrels    11
15,001 to 999,999 Barrels         11
6,000,001 Barrels and Over        11
Under 1 Barrel                     6
Name: count, dtype: int64

##### Our focus is not on breweries that are not in production stages, like in the early brewery planning stages or breweries in the final closure stages. Also, this 'Under 1 Barrel' special circumstances category is missing in multiple of the years and so it does not  add  any significant value to the visualization. As such, we are removing the '"Under 1 Barrel" category all together below.

In [323]:
# brewer_size_df"Under 1 Barrel"
# print csv dataset
print('*'*80,"\nbrewer_size_df:\n")
print(f"\nRows before removing 'Under 1 Barrel' rows: {df_dict['brewer_size'].shape[0]}\n")
if df_dict['brewer_size'][df_dict['brewer_size']['brewer_size'].astype(str).str.contains('Under 1 Barrel', case=False)].shape[0] > 0:
    print(f"\nsample of 'Under 1 Barrel' labels in '{'brewer_size'}' column:\n")
    print(df_dict['brewer_size'][df_dict['brewer_size']['brewer_size'].astype(str).str.contains('Under 1 Barrel', case=False)]['brewer_size'].head(4))
    df_dict['brewer_size'] = df_dict['brewer_size'][~df_dict['brewer_size']['brewer_size'].astype(str).str.contains('Under 1 Barrel', case=False)]            
print(f"\nRows after removing any 'Under 1 Barrel' rows: {df_dict['brewer_size'].shape[0]}\n")
df_dict['brewer_size']['brewer_size'].value_counts()

******************************************************************************** 
brewer_size_df:


Rows before removing 'Under 1 Barrel' rows: 50


sample of 'Under 1 Barrel' labels in 'brewer_size' column:

4     Under 1 Barrel
29    Under 1 Barrel
34    Under 1 Barrel
39    Under 1 Barrel
Name: brewer_size, dtype: object

Rows after removing any 'Under 1 Barrel' rows: 44



brewer_size
1 to 15,000 Barrels               11
1,000,000 to 6,000,000 Barrels    11
15,001 to 999,999 Barrels         11
6,000,001 Barrels and Over        11
Name: count, dtype: int64

##### In 'brewing_materials' df, grouping subtotals by 'year' & removing 'month' column.  Also, removing higher subtotal level 'material_type' category.

In [324]:
# summing the categories by Grouping 
df_material_grouped = df_dict["brewing_materials"][['year','type','month_current']].groupby(['year', 'type']).sum().reset_index()
print(f"Difference between 'month_current': {round(df_dict['brewing_materials']['month_current'].sum() - df_material_grouped['month_current'].sum(),2):,}",'\n\n')
print(df_material_grouped.head(3))

# Setting 'brewing_materials' in df dictionary to new version. 
df_dict['brewing_materials'] = df_material_grouped

print('\n\n',df_dict['brewing_materials'].head(3))

Difference between 'month_current': 0 


   year                        type  month_current
0  2008  Barley and barley products       58048335
1  2008      Corn and corn products      726766392
2  2008                  Hops (dry)       54889519


    year                        type  month_current
0  2008  Barley and barley products       58048335
1  2008      Corn and corn products      726766392
2  2008                  Hops (dry)       54889519


##### In 'beer_states' df, grouping subtotals by 'year' & removing 'type' column. 

In [325]:
# Add percentage_change to df_dict['brewing_materials']
df_material_percentage = df_dict['brewing_materials'].sort_values(by=['type', 'year'])


df_material_percentage['percentage_change'] = df_material_percentage.groupby('type')['month_current'].pct_change()

# Convert the percentage change values to percentage format
df_material_percentage['percentage_change'] = round(df_material_percentage['percentage_change'].fillna(0) * 100,2)

print(df_material_percentage[df_material_percentage['type'] == 'Malt and malt products'].head(10))

print(f"Difference between 'brewing_materials': {round(df_dict['brewing_materials']['month_current'].sum() - df_material_percentage['month_current'].sum(),2):,}")


# Setting 'brewing_materials' in df dictionary to new version. 
df_dict['brewing_materials'] = df_material_percentage

    year                    type  month_current  percentage_change
4   2008  Malt and malt products     4497468603               0.00
13  2009  Malt and malt products     4298874888              -4.42
22  2010  Malt and malt products     4146613431              -3.54
31  2011  Malt and malt products     4003433125              -3.45
40  2012  Malt and malt products     4099141038               2.39
49  2013  Malt and malt products     3939110211              -3.90
58  2014  Malt and malt products     3790022536              -3.78
67  2015  Malt and malt products     3669211565              -3.19
76  2016  Malt and malt products      141013476             -96.16
85  2017  Malt and malt products      112425009             -20.27
Difference between 'brewing_materials': 0


##### Please note the 'brewing_materials' dataset 96% drop off in 2016 above. In 2015 the TTB filings used for this dataset change the form to face out the beer production _material reporting requirement_. The 'brewing_materials' data goes to nearly zero by year 2016.  

##### In 'beer_states' df, grouping subtotals by 'state' & removing 'type' column. 

In [326]:
# summing the categories by Grouping 
df_state_grouped = df_dict["beer_states"][['state','year','barrels']].groupby(['year', 'state']).sum().reset_index()

print(f"Difference between 'barrels': {round(df_dict['beer_states']['barrels'].sum() - df_state_grouped['barrels'].sum(),2):,}")


# Setting 'brewing_materials' in df dictionary to new version. 
df_dict['beer_states'] = df_state_grouped

print('\n\n',df_dict['beer_states'].head(3))

Difference between 'barrels': 0.0


    year state       barrels
0  2008    AK  1.342068e+05
1  2008    AL  1.166716e+06
2  2008    AR  3.417910e+03


##### In 'beer_states' df, getting  percentage of 'barrels' for each 'state' for better map representation. 

In [327]:
# Add percentage_change to df_dict['brewing_materials']
df_dict['beer_states']['barrels']  = round(df_dict['beer_states']['barrels'],2)

# calculate the total 'barrels' for each 'year'
df_dict['beer_states']['total_barrels_per_year'] = df_dict['beer_states'].groupby('year')['barrels'].transform('sum')

# Calculate the percentage of 'barrels' for each 'state' in each 'year'
df_dict['beer_states']['percentage_total_barrels'] = round(df_dict['beer_states']['barrels'] / df_dict['beer_states']['total_barrels_per_year'] * 100,2)

df_dict["beer_states"] = df_dict["beer_states"][['state','year','barrels','percentage_total_barrels']]


print(df_dict["beer_states"].head(10))

  state  year      barrels  percentage_total_barrels
0    AK  2008    134206.79                      0.07
1    AL  2008   1166715.84                      0.61
2    AR  2008      3417.91                      0.00
3    AZ  2008     40474.66                      0.02
4    CA  2008  22149408.23                     11.56
5    CO  2008  26795857.25                     13.98
6    CT  2008      8043.24                      0.00
7    DC  2008   1169525.08                      0.61
8    DE  2008     85091.87                      0.04
9    FL  2008   9497348.84                      4.95


##### Adding State names from State abbreviations to Names in beer_states_df

In [328]:
# USA States Dictionary 
us_state_abbrev = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

# Add a new column 'state_name' by mapping the 'state_abbr' column using the dictionary
df_dict['beer_states']['state_name'] = df_dict['beer_states']['state'].map(us_state_abbrev)

# USA States id Dictionary for us-10m.json
us_state_id_dict = { "AL": "1",  "AK": "2",  "AZ": "4",  "AR": "5",  "CA": "6",  "CO": "8",  "CT": "9",  "DE": "10",  "DC": "11",  "FL": "12",  "GA": "13",  "HI": "15",  "ID": "16",  "IL": "17",  "IN": "18",  "IA": "19",  "KS": "20",  "KY": "21",  "LA": "22",  "ME": "23",  "MD": "24",  "MA": "25",  "MI": "26",  "MN": "27",  "MS": "28",  "MO": "29",  "MT": "30",  "NE": "31",  "NV": "32",  "NH": "33",  "NJ": "34",  "NM": "35",  "NY": "36",  "NC": "37",  "ND": "38",  "OH": "39",  "OK": "40",  "OR": "41",  "PA": "42",  "RI": "44",  "SC": "45",  "SD": "46",  "TN": "47",  "TX": "48",  "UT": "49",  "VT": "50",  "VA": "51",  "WA": "53",  "WV": "54",  "WI": "55",  "WY": "56"}

# Add a new column 'id' by mapping the 'state_abbr' column using the States id dictionary
df_dict['beer_states']['id'] = df_dict['beer_states']['state'].map(us_state_id_dict).astype(str)

# Show  'state_name' & 'id' value for each 'state_abbr' value 
df_dict['beer_states'][['state', 'state_name', 'id']].groupby(['state','state_name']).value_counts()

state  state_name            id
AK     Alaska                2     12
AL     Alabama               1     12
AR     Arkansas              5     12
AZ     Arizona               4     12
CA     California            6     12
CO     Colorado              8     12
CT     Connecticut           9     12
DC     District of Columbia  11    12
DE     Delaware              10    12
FL     Florida               12    12
GA     Georgia               13    12
HI     Hawaii                15    12
IA     Iowa                  19    12
ID     Idaho                 16    12
IL     Illinois              17    12
IN     Indiana               18    12
KS     Kansas                20    12
KY     Kentucky              21    12
LA     Louisiana             22    12
MA     Massachusetts         25    12
MD     Maryland              24    12
ME     Maine                 23    12
MI     Michigan              26    12
MN     Minnesota             27    12
MO     Missouri              29    12
MS     Mississippi

##### Save to csv's

In [329]:
# Create function to save each df to csv
import csv


def save_to_csv(d):
    for name, df in d.items():
        print("_"*80,f"\n{name}.csv:\n")
        df.to_csv(f'{name}.csv',index=False, quoting=csv.QUOTE_NONNUMERIC)

# Run each df

save_to_csv(df_dict)
print("_"*80,f"\n{'beer_states'}.csv:\n")
df_dict['beer_states'].to_csv(f'{"beer_states"}.csv',index=False, quoting=csv.QUOTE_NONNUMERIC)

________________________________________________________________________________ 
beer_taxed.csv:

________________________________________________________________________________ 
beer_states.csv:

________________________________________________________________________________ 
brewer_size.csv:

________________________________________________________________________________ 
brewing_materials.csv:

________________________________________________________________________________ 
beer_states.csv:

